In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
from denn import *
import pandas as pd
import io #Used as buffer

In [2]:
path= Path(f'../../data/cluster_results')

In [4]:
def read_csv(f):
    df = pd.read_csv(f).mean()
    df['experiment'] = f.parent.parent.parent.name
    df['function'] = f.parent.parent.name
    name = f.parent.name
    df['freq'] = int(name[4:].split('n')[0])
    if 'noNN' in f.name:
        df['method'] = f.name.split('_')[0]       
    else:
        df['method'] = f.name.split('_')[0]+'_'+f.name.split('_')[1]
    return df

measure_name=['mof','sr','arr','nfe','nn_time']
index_measure=0

In [ ]:
m = measure_name[index_measure]
files  = list(path.glob(f'exp*/*/freq1*nn_s2nn_tw5*bs4*/*{m}.csv'))
# files += list(path.glob(f'exp*/*/*nn_p3/*{m}.csv'))
files += [f for f in path.glob(f'exp*/*/freq1*nn_p3/*{m}.csv') if 'noNN' in f.name]
#print(files)
df = pd.DataFrame([read_csv(f) for f in files])
# data = pd.pivot_table(df, index=['experiment','function'], columns='method', values=m) #,'freq' later for adding frequency experiment
data = pd.pivot_table(df, index=['experiment', 'freq','function'],  columns='method') #,'freq' later for adding frequency experiment, ,values=m
data.style.format("{:,.2f}")

In [46]:
# with std
def read_csv(f):
    df = pd.read_csv(f).mean()
    df['std'] = pd.read_csv(f).std().iloc[0]
    df['experiment'] = f.parent.parent.parent.name
    df['function'] = f.parent.parent.name
    name = f.parent.name
    df['freq'] = int(name[4:].split('n')[0])
    if 'noNN' in f.name:
        df['method'] = f.name.split('_')[0]       
    else:
        df['method'] = f.name.split('_')[0]+'_'+f.name.split('_')[1]
    return df

measure_name=['mof','sr','arr','nfe','nn_time']
index_measure=0

In [47]:
m = measure_name[index_measure]
h=['noNN','Closest']
files  = list(f for f in path.glob(f'exp*/*/freq1*nn_s2nn_tw5*bs4*/*{m}.csv') if 'noNN' not in f.name and 'Closest' not in f.name)
# files += list(path.glob(f'exp*/*/*nn_p3/*{m}.csv'))
#files += [f for f in path.glob(f'exp*/*/freq1*nn_p3/*{m}.csv') if 'noNN' in f.name]
#print(files)
df = pd.DataFrame([read_csv(f) for f in files])
# data = pd.pivot_table(df, index=['experiment','function'], columns='method', values=m) #,'freq' later for adding frequency experiment
data = pd.pivot_table(df, index=[ 'experiment','function','freq'],  columns='method') #,'freq' later for adding frequency experiment, ,values=m
data.style.format("{:,.2f}")

In [48]:
final_data = data[m].copy()

for col in data[m].columns:
    final_data[col] = data[m][col].apply(lambda x: f'{x:,.2f} (±') + data['std'][col].apply(lambda x: f'{x:,.2f})')

In [40]:
final_data

method                                      NNdrop_Random  \
experiment function   freq                                  
exp1       rastrigin  1                     21.33 (±3.68)   
           rosenbrock 1          253,692.23 (±151,509.51)   
           sphere     1                     13.27 (±6.87)   
exp2       rastrigin  1                      1.82 (±0.17)   
           rosenbrock 1                   130.58 (±35.87)   
           sphere     1                      0.19 (±0.05)   
exp3       rastrigin  1                     15.77 (±4.33)   
           rosenbrock 1                4,166.39 (±112.07)   
           sphere     1                      5.28 (±0.11)   
exp4       rastrigin  1                     56.20 (±3.41)   
           rosenbrock 1     16,191,014.43 (±2,191,672.49)   
           sphere     1                     59.04 (±4.85)   

method                                       NNdrop_Worst  \
experiment function   freq                                  
exp1       rastrigin  1                     22.21 (±2.89)   
           rosenbrock 1          157,765.36 (±168,332.93)   
           sphere     1                      9.58 (±7.55)   
exp2       rastrigin  1                      1.87 (±0.21)   
           rosenbrock 1                   114.44 (±46.30)   
           sphere     1                      0.20 (±0.06)   
exp3       rastrigin  1                     13.09 (±4.82)   
           rosenbrock 1                 4,190.50 (±32.91)   
           sphere     1                      5.10 (±0.97)   
exp4       rastrigin  1                     57.44 (±3.29)   
           rosenbrock 1     12,911,541.56 (±2,193,540.47)   
           sphere     1                     55.87 (±4.76)   

method                                      NNnorm_Random  \
experiment function   freq                                  
exp1       rastrigin  1                     15.47 (±5.47)   
           rosenbrock 1            56,145.15 (±51,920.09)   
           sphere     1                      5.80 (±3.67)   
exp2       rastrigin  1                      2.21 (±0.23)   
           rosenbrock 1                     48.69 (±4.76)   
           sphere     1                      0.09 (±0.00)   
exp3       rastrigin  1                     13.53 (±3.04)   
           rosenbrock 1                3,062.63 (±937.19)   
           sphere     1                      4.57 (±0.82)   
exp4       rastrigin  1                     93.60 (±4.67)   
           rosenbrock 1     13,182,851.37 (±2,424,159.15)   
           sphere     1                     45.33 (±5.01)   

method                                      NNnorm_Worst  
experiment function   freq                                
exp1       rastrigin  1                    17.33 (±5.19)  
           rosenbrock 1           47,610.23 (±63,381.53)  
           sphere     1                     6.22 (±4.60)  
exp2       rastrigin  1                     2.10 (±0.20)  
           rosenbrock 1                    50.27 (±4.22)  
           sphere     1                     0.10 (±0.01)  
exp3       rastrigin  1                    13.69 (±3.52)  
           rosenbrock 1             3,026.08 (±1,226.57)  
           sphere     1                     4.45 (±0.91)  
exp4       rastrigin  1                    59.41 (±3.49)  
           rosenbrock 1     8,951,062.09 (±1,545,930.92)  
           sphere     1                    43.95 (±3.97)

In [49]:
print(final_data.to_csv(float_format='%.2f'))

experiment,function,freq,NNdrop_Random,NNdrop_Worst,NNnorm_Random,NNnorm_Worst
exp1,rastrigin,1,21.33 (±3.68),22.21 (±2.89),15.47 (±5.47),17.33 (±5.19)
exp1,rosenbrock,1,"253,692.23 (±151,509.51)","157,765.36 (±168,332.93)","56,145.15 (±51,920.09)","47,610.23 (±63,381.53)"
exp1,sphere,1,13.27 (±6.87),9.58 (±7.55),5.80 (±3.67),6.22 (±4.60)
exp2,rastrigin,1,1.82 (±0.17),1.87 (±0.21),2.21 (±0.23),2.10 (±0.20)
exp2,rosenbrock,1,130.58 (±35.87),114.44 (±46.30),48.69 (±4.76),50.27 (±4.22)
exp2,sphere,1,0.19 (±0.05),0.20 (±0.06),0.09 (±0.00),0.10 (±0.01)
exp3,rastrigin,1,15.77 (±4.33),13.09 (±4.82),13.53 (±3.04),13.69 (±3.52)
exp3,rosenbrock,1,"4,166.39 (±112.07)","4,190.50 (±32.91)","3,062.63 (±937.19)","3,026.08 (±1,226.57)"
exp3,sphere,1,5.28 (±0.11),5.10 (±0.97),4.57 (±0.82),4.45 (±0.91)
exp4,rastrigin,1,56.20 (±3.41),57.44 (±3.29),93.60 (±4.67),59.41 (±3.49)
exp4,rosenbrock,1,"16,191,014.43 (±2,191,672.49)","12,911,541.56 (±2,193,540.47)","13,182,851.37 (±2,424,159.15)","8,951,062.09 (±1

In [ ]:
!pwd

In [ ]:
data.to_csv('tmp.csv', float_format='%.2f')

In [ ]:
!rm tmp.csv

In [ ]:
Func_names=['sphere','rastrigin','rosenbrock']
exp_names=['exp1','exp2','exp3','exp4']
frequency=1000
nn_window=5


In [ ]:
#Experiment = Enum('Experiment', 'exp1 exp2 exp3 exp4')
#Method = Enum('Methods', 'noNNRestart noNNReval NNnorm NNdrop')
#Mechname:Random, Worst, Closest
measure_name=['mof','sr','arr','nfe','nn_time']
index_measure=0
total=[]
#lables = ['noNNReval', 'NNnorm_Random', 'NNnorm_Worst','NNnorm_Closest','NNdrop_Random', 'NNdrop_Worst', 'NNdrop_Closest']
lables = ['noNNReval', 'NNnorm_Worst', 'NNdrop_Worst']

for experiment in exp_names:
    
    for func in Func_names:
       # path = Path(f'../../data/results/{experiment}/{func}')
        path_in = Path(f'../../data/cluster_results/{experiment}/{func}')
        path = path_in / f'freq{frequency}nn_w{nn_window}nn_p{3}'
        noNNReval= pd.read_csv(path/f'noNNReval_{measure_name[index_measure]}.csv').mean(0).values[0]

        #noNNRestart= pd.read_csv(path/'noNNRestart_mof.csv')

       # NNnorm_Random= pd.read_csv(path/f'NNnorm_Worst_{measure_name[index_measure]}.csv').mean(0).values[0]
       # NNdrop_Random = pd.read_csv(path/f'NNdrop_Worst_{measure_name[index_measure]}.csv').mean(0).values[0]


        NNnorm_Worst= pd.read_csv(path/f'NNnorm_Worst_{measure_name[index_measure]}.csv').mean(0).values[0]
        NNdrop_Worst = pd.read_csv(path/f'NNdrop_Worst_{measure_name[index_measure]}.csv').mean(0).values[0]


       # NNnorm_Closest= pd.read_csv(path/f'NNnorm_Closest_{measure_name[index_measure]}.csv').mean(0).values[0]
       # NNdrop_Closest = pd.read_csv(path/f'NNdrop_Closest_{measure_name[index_measure]}.csv').mean(0).values[0]
        
       # x=np.array([noNNReval.mof, NNnorm_Random.mof, NNnorm_Worst.mof, NNnorm_Closest.mof, NNdrop_Random.mof,NNdrop_Worst.mof,NNdrop_Closest.mof])
        #y=[noNNReval, NNnorm_Random,NNnorm_Worst,NNnorm_Closest,NNdrop_Random, NNdrop_Worst, NNdrop_Closest]
        y=[noNNReval, NNnorm_Worst,NNdrop_Worst]

        total.append(y)
        #print(total)

    #,  index=[Func_names,Func_names]

#df=pd.DataFrame(total, columns=lables,index=['sphere','rastrigin','rosenbrock','sphere','rastrigin','rosenbrock'])
df=pd.DataFrame(total, columns=lables,index=['sphere','rastrigin','rosenbrock','sphere','rastrigin','rosenbrock','sphere','rastrigin','rosenbrock', 'sphere','rastrigin','rosenbrock'])

#pd.options.display.float_format = '{:,.2f}'.format


print(pd.DataFrame.to_csv(df, float_format='%.2f'))


#pd.DataFrame.to_latex(df, multirow=True)    #, float_format="%%.2f" ,, , longtable=True,  bold_rows=True,


In [ ]:
pd.read_csv('tmp.csv')

In [ ]:
df

In [ ]:
#report nn_time
measure_name=['mof','sr','arr','nfe','nn_time']
index_measure=4
total=[]
lables = [ 'NNnorm_Random', 'NNnorm_Worst','NNnorm_Closest','NNdrop_Random', 'NNdrop_Worst', 'NNdrop_Closest']

for experiment in exp_names:
    
    for func in Func_names:
        path = Path(f'../../data/results/{experiment}/{func}')

        NNnorm_Random= pd.read_csv(path/f'NNnorm_Worst_{measure_name[index_measure]}.csv').mean(0).values[0]
        NNdrop_Random = pd.read_csv(path/f'NNdrop_Worst_{measure_name[index_measure]}.csv').mean(0).values[0]


        NNnorm_Worst= pd.read_csv(path/f'NNnorm_Worst_{measure_name[index_measure]}.csv').mean(0).values[0]
        NNdrop_Worst = pd.read_csv(path/f'NNdrop_Worst_{measure_name[index_measure]}.csv').mean(0).values[0]

        NNnorm_Closest= pd.read_csv(path/f'NNnorm_Closest_{measure_name[index_measure]}.csv').mean(0).values[0]
        NNdrop_Closest = pd.read_csv(path/f'NNdrop_Closest_{measure_name[index_measure]}.csv').mean(0).values[0]
        
       # x=np.array([noNNReval.mof, NNnorm_Random.mof, NNnorm_Worst.mof, NNnorm_Closest.mof, NNdrop_Random.mof,NNdrop_Worst.mof,NNdrop_Closest.mof])
        y=[ NNnorm_Random,NNnorm_Worst,NNnorm_Closest,NNdrop_Random, NNdrop_Worst, NNdrop_Closest]
        total.append(y)
        #print(total)


df=pd.DataFrame(total, columns=lables,index=['sphere','rastrigin','rosenbrock','sphere','rastrigin','rosenbrock'])



print(pd.DataFrame.to_csv(df, float_format='%.2f'))
#df
